<a href="https://colab.research.google.com/github/aldebaro/ai6g/blob/main/03_channel_estimation_hybrid_mimo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Inteligência Artificial e Aprendizado de Máquina Aplicados a Redes 5G e 6G**.
*Aldebaro Klautau* (UFPA). Minicurso 5 do SBrT - 25 de setembro de 2022.
Authors: XXX

To install and import all the Needed packages

In [ ]:
# Clone the repository if running in Colab and install all the dependencies
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive/')
    import sys
    import os
    try:
      !git clone https://github.com/aldebaro/ai6g.git
    except:
      print("ai6g is already in the contents")
    %cd ai6g
    !ln -s /content/drive/MyDrive/ai6g_files/files_03_channel/* ./files_03_channel
    !pip install -r requirements.txt

In [42]:
import h5py 
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    Dense,
    Reshape,
    Conv2D,
    Dropout,
    Flatten,
    BatchNormalization,
)
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
)


In [43]:

FILES={
    1:"./files_03_channel/Pilots_SNR_(-5dB)_64x16(80pilots)_single_sub.hdf5",
    2:"./files_03_channel/Pilots_SNR_(-5dB)_8x4(80pilots)_single_sub.hdf5",
    3:"./files_03_channel/Pilots_SNR_(0dB)_8x4(80pilots)_single_sub.hdf5",
    "64x16":"./files_03_channel/Channels_64x16_single_sub.hdf5",
    "8x4":"./files_03_channel/Channels_8x4_single_sub.hdf5",
}


Change the following values according to the dataset

In [44]:
CHOSEN_DATASET = 2
BATCH_SIZE = 32
EPOCHS = 50 #original value 1000


# Change these values acording to the dataset
NR = 8 
NT = 4 
LR = 4 
CHANNELS_USED_TRAIN = 9000
SUBCARIERS_USED = 1
TOTAL_CHANNELS = 10000
NUM_SYM_PILOTS = 80
CHANNELS_USED_TEST = TOTAL_CHANNELS - CHANNELS_USED_TRAIN

trained_model = "./files_03_channel/outputs/model.h5" #output file with neural net model

input_shape = (NUM_SYM_PILOTS * LR, SUBCARIERS_USED)
output_shape = (NR, NT, SUBCARIERS_USED)

SELECTING THE DATASET

In [45]:
if CHOSEN_DATASET == 1:
    CHOSEN_CHANNELS = FILES["64x16"]
elif CHOSEN_DATASET > 1 and CHOSEN_DATASET <=3:
    CHOSEN_CHANNELS = FILES["8x4"]


# Directory of the Channels Matrix file
CHANNELS_FILE = h5py.File(CHOSEN_CHANNELS, "r")


# Directory of the Pilots Matrix file
PILOTS_FILE = h5py.File(FILES[CHOSEN_DATASET], "r")

This function will be used as a metric in keras to calculate the NMSE(Normalized Mean Squared Error) for each train or validation example that will be iterated in the training process, providing clear metrics to the whole process and enabling the use of early_stopping.

In [46]:
# Declaring the function to calculate NMSE for each batch
def NMSEtrainComplex(y_true, y_pred):
    sub = y_pred[:, :, :] - y_true[:, :, :]
    H_k = y_true[:, :, :]
    nmse = tf.norm(sub, ord="fro", axis=(1, 2)) ** 2
    den = tf.norm(H_k, ord="fro", axis=(1, 2)) ** 2

    result = (nmse / den)

    return 10*tf.experimental.numpy.log10(result)

Now the NN model will be crated, a chain of Dense layers is being used with relu activation.

In [47]:
model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(BatchNormalization())
model.add(Dense((512), activation="relu"))
model.add(Dropout(0.1))
model.add(Dense((512), activation="relu"))
model.add(Dropout(0.1))
#disable the layers below to save computational cost
#model.add(Dense((512), activation="relu"))
#model.add(Dropout(0.1))
#model.add(Dense((512), activation="relu"))
#model.add(Dropout(0.1))
#model.add(Dense((512), activation="relu"))
#model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(np.prod(output_shape), activation="linear"))
model.add(Reshape(output_shape))
model.compile(
    loss="mean_squared_error", optimizer="adam", metrics=[NMSEtrainComplex]
    )
print(model.summary()) #show the network topology

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 320)               0         
                                                                 
 batch_normalization_12 (Bat  (None, 320)              1280      
 chNormalization)                                                
                                                                 
 dense_36 (Dense)            (None, 512)               164352    
                                                                 
 dropout_30 (Dropout)        (None, 512)               0         
                                                                 
 dense_37 (Dense)            (None, 512)               262656    
                                                                 
 dropout_31 (Dropout)        (None, 512)               0         
                                                      

Train the Model

In [48]:

training_inputData = PILOTS_FILE["pilots"][:CHANNELS_USED_TRAIN, :, :SUBCARIERS_USED]
training_outputData = CHANNELS_FILE["channels"][:CHANNELS_USED_TRAIN, :, :, :SUBCARIERS_USED]


history = model.fit(
        training_inputData,
        training_outputData,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        verbose=1,
        shuffle=True,
        validation_split=0.2,
        callbacks=[
            EarlyStopping(
                monitor="val_NMSEtrainComplex",
                min_delta=5e-3,
                patience=40,
                restore_best_weights=True,
            ),
            ReduceLROnPlateau(
                factor=0.5,
                min_delta=5e-2,
                patience=20,
                cooldown=5,
                verbose=1,
                min_lr=1e-7,
            ),
        ],
    )
model.save(trained_model)

Epoch 1/50
225/225 [==============================] - 2s 7ms/step - loss: 0.7860 - NMSEtrainComplex: 0.6141 - val_loss: 0.3159 - val_NMSEtrainComplex: -3.0464 - lr: 0.0010
Epoch 2/50
225/225 [==============================] - 1s 6ms/step - loss: 0.3653 - NMSEtrainComplex: -2.3629 - val_loss: 0.2584 - val_NMSEtrainComplex: -4.0090 - lr: 0.0010
Epoch 3/50
225/225 [==============================] - 1s 5ms/step - loss: 0.2890 - NMSEtrainComplex: -3.4071 - val_loss: 0.2401 - val_NMSEtrainComplex: -4.3724 - lr: 0.0010
Epoch 4/50
225/225 [==============================] - 1s 5ms/step - loss: 0.2544 - NMSEtrainComplex: -3.9940 - val_loss: 0.2317 - val_NMSEtrainComplex: -4.5368 - lr: 0.0010
Epoch 5/50
225/225 [==============================] - 1s 5ms/step - loss: 0.2350 - NMSEtrainComplex: -4.3519 - val_loss: 0.2242 - val_NMSEtrainComplex: -4.6522 - lr: 0.0010
Epoch 6/50
225/225 [==============================] - 1s 5ms/step - loss: 0.2226 - NMSEtrainComplex: -4.5813 - val_loss: 0.2215 - val_NM

Epoch 47/50
225/225 [==============================] - 1s 5ms/step - loss: 0.0865 - NMSEtrainComplex: -8.7635 - val_loss: 0.1527 - val_NMSEtrainComplex: -6.6682 - lr: 5.0000e-04
Epoch 48/50
225/225 [==============================] - 1s 5ms/step - loss: 0.0850 - NMSEtrainComplex: -8.8390 - val_loss: 0.1522 - val_NMSEtrainComplex: -6.6911 - lr: 5.0000e-04
Epoch 49/50
225/225 [==============================] - 1s 5ms/step - loss: 0.0870 - NMSEtrainComplex: -8.7328 - val_loss: 0.1529 - val_NMSEtrainComplex: -6.6796 - lr: 5.0000e-04
Epoch 50/50
225/225 [==============================] - 1s 6ms/step - loss: 0.0854 - NMSEtrainComplex: -8.8210 - val_loss: 0.1523 - val_NMSEtrainComplex: -6.7056 - lr: 5.0000e-04


TEST MODEL

In [49]:
#need to inform about the custom metric, to allow it to be incorporated in the file
print("Reading trained model from", trained_model)
mlModel = load_model(trained_model, custom_objects={"NMSEtrainComplex": NMSEtrainComplex})

TEST_INPUT = PILOTS_FILE["pilots"][CHANNELS_USED_TRAIN:, :, :SUBCARIERS_USED]
TEST_OUTPUT = CHANNELS_FILE["channels"][CHANNELS_USED_TRAIN:, :, :, :SUBCARIERS_USED]

results = mlModel.evaluate(TEST_INPUT,TEST_OUTPUT)
print("Loss and NMSE (in dB):", results)





Reading trained model from ./files_03_channel/outputs/model.h5
32/32 [==============================] - 0s 4ms/step - loss: 0.1456 - NMSEtrainComplex: -6.7385
Loss and NMSE (in dB): [0.1455831229686737, -6.738473892211914]


## Authors

- Prof. Francisco Muller - LASSE/UFPA
- Daniel Oliveira - LASSE/UFPA
- Claudio Mello - LASSE/UFPA